In [2]:
from pathlib import Path
import json

In [3]:
input_json_file = Path("./data.json")
output_json_file = Path("./transform.json")

In [5]:
with open(input_json_file, "r") as infile:
    ego_data = json.load(infile)

In [11]:
neus2_json = {
                "w": 3840,
                "h": 2160,
                "aabb_scale": 1,
                "scale": 1,
                "offset": [
                    0,
                    0,
                    0
                ],
                "from_na": True,
                "frames": []
             }

In [12]:
all_cameras = ["gp01_0", "gp02_0", "gp03_0", "gp04_0", "gp05_0"]

In [23]:
width = 3840
height = 2160

In [24]:
for camera in all_cameras:
    cam_data = ego_data[camera]
    frame_path = cam_data["frame_path"]
    world_to_cam = cam_data["camera_data"]["T_world_device"]
    extrinsics_list = world_to_cam
    
    # Fisheye camera intrinsics
    # [fx, fy, cx, cy, k1, k2, k3, k4]
    fx = cam_data["_raw_camera"]["intrinsics_0"]
    fy = cam_data["_raw_camera"]["intrinsics_1"]
    cx = cam_data["_raw_camera"]["intrinsics_2"]
    cy = cam_data["_raw_camera"]["intrinsics_3"]
    k1 = cam_data["_raw_camera"]["intrinsics_4"]
    k2 = cam_data["_raw_camera"]["intrinsics_5"]
    k3 = cam_data["_raw_camera"]["intrinsics_6"]
    k4 = cam_data["_raw_camera"]["intrinsics_7"]

    # For Undistortion
    # TODO

    # For NeuS2
    intrinsics_list = [
        [   fx,
            0.0,
            width - cx,
            0.0
        ],
        [
            0.0,
            fy,
            height - cy,
            0.0
        ],
        [
            0.0,
            0.0,
            1.0,
            0.0
        ],
        [
            0.0,
            0.0,
            0.0,
            1.0
        ]
    ]
    
    frame_data = {}
    frame_data["file_path"] = frame_path
    frame_data["transform_matrix"] = extrinsics_list
    frame_data["intrinsic_matrix"] = intrinsics_list
    
    neus2_json["frames"].append(frame_data)


In [25]:
neus2_json_object = json.dumps(neus2_json, indent=4)
with open(output_json_file, "w") as outfile:
    outfile.write(neus2_json_object)